**what types of free app attract most users?**

***metrics*** (rated in orders)  
 1) Price == 0  
 2) Install > 10000  
 3) Genres  
 4) Users_ratings_counts


In [1]:
def open_file(file_name):
    data = open(file_name, encoding = "utf8")
    from csv import reader
    read_data = reader(data)
    return list(read_data)

App_data = open_file('AppleStore.csv')

Google_data = open_file('googleplaystore.csv')

In [2]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
explore_data(Google_data, 0, 3, True)
android = Google_data[1:]
ios = App_data[1:]

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


***metric keywords:***  
1) Price  
2) Installs  
3) Category/Genre (index 9)  
4) Reviews  
[preferences](https://www.kaggle.com/lava18/google-play-store-apps/home#googleplaystore.csv)

In [3]:
explore_data(App_data, 0, 3)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']




***metric keywords:***  
1) price  
2) raing_count_tot  
3) prime_genre (index 11)

[preferences](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)
  

In [4]:
print(len(android))
del android[10472]
print(len(android))

10841
10840


In [5]:
def duplicate_data(data_set):
    duplicate_apps = []
    unique_apps = []
    for app in data_set[1:]:
        name = app[0]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    return duplicate_apps

In [6]:
duplicate_Google = duplicate_data(Google_data)
print(duplicate_Google[0:8])

['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads']


In [7]:
len(duplicate_Google)

1181

**Strategy for removing duplicate**  

> Keep the row with the higesht number of reviews and remove the other entries. The higher number of reviews, the more recent the data should be.

In [8]:
reviews_max = {}
for app in android:
        name = app[0]
        n_review = float(app[3])
        if name in reviews_max and reviews_max[name] < n_review:
            reviews_max[name] = n_review
        elif name not in reviews_max:
            reviews_max[name] = n_review
print('Expected length:',  len(android) - 1181)
print('Actual length:',  len(reviews_max))

Expected length: 9659
Actual length: 9659


Extracting the duplication by using reviews number. If there are multiple duplicates of a same app, extract the data with the highest review.   

> We first creat a library name *review_max* and a loop to run through android data set which takes name and reviews of apps.  

> If *name* of the app already in the *reviews_max* and the review is lower than the new value   
`` reviews_max[name] < n_reviews``  
We update library  

> if *name* of the app is not in the libary list, we create a new key and put int the new value.

In [9]:
android_clean =[]
already_added = []
for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
        
len(android_clean)

9659

This is built upon the duplicate data set we created. We loop through data set with 2 variables *name* and *n_reviews*, our filter here is:  

>```reviews_max[name] == n_reiews```  

If the above is true, we add the entire row into android_clean data set and the name of the app into already_added variable to check for replicate.


In [10]:
def IsThisEnglish(string):
    non_standard_English = 0
    for character in string:
        if ord(character) > 127:
            non_standard_English += 1
            
        if non_standard_English > 3:
            return False
        
    return True
IsThisEnglish('Instachat 😜')

android_English = []
ios_English = []

for app in android_clean:
    name = app[0]
    if IsThisEnglish(name):
        android_English.append(app)
for app in ios:
    name = app[1]
    if IsThisEnglish(name):
        ios_English.append(app)
        
print(len(android_English))
print('\n')
print(len(ios_English))

9614


6183


In [11]:
free_android = []
free_ios = []
for app in android_English:
    price = app[7]
    if price == '0':
        free_android.append(app)
for app in ios_English:
    price = app[4]
    if price == '0.0':
        free_ios.append(app)
print(len(free_android))
print('\n')
print(len(free_ios))

    
        

8864


3222


sorted function doesnt work well with with dictionaries => transform dictionary into tuple function. 
```Python
freq_table = {'Genre_1': 50, 'Genre_2': 100}
sorted(freq_table)
==>
> freq_table_as_tuple =[(50, 'Genre_1'), (100, 'Genre_2')]
```

In [12]:
def freq_table(dataset, index):
    table ={}
    total = 0
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
    table_percentage ={}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentage[key]= percentage
    return table_percentage
    
android_frequency = freq_table(free_android, 9)
    

In [13]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [14]:
android_final = display_table(free_android, 1)#volume of apps in Google store by category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [15]:
ios_final = display_table(free_ios, -5) #volume of apps in App store by genres

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [16]:
#Numbers of download (calculate with user ratings) categorized by genres in app store
genres_ios ={}
for row in free_ios:
    genres_app = row[-5]
    if genres_app in genres_ios:
        genres_ios[genres_app] += 1
    else:
        genres_ios[genres_app] = 1

for genre in genres_ios:
    total = 0
    len_total = 0
    genre_table = {}
    for app in free_ios:
        genre_app = app[-5]
        if genre == genre_app:
            n_ratings = float(app[5])
            total += n_ratings
            len_total += 1
    avg_n_ratings = total/ len_total
    print(genre, ':' , avg_n_ratings)
    

Catalogs : 4004.0
Travel : 28243.8
Finance : 31467.944444444445
News : 21248.023255813954
Utilities : 18684.456790123455
Education : 7003.983050847458
Book : 39758.5
Reference : 74942.11111111111
Photo & Video : 28441.54375
Food & Drink : 33333.92307692308
Productivity : 21028.410714285714
Business : 7491.117647058823
Shopping : 26919.690476190477
Lifestyle : 16485.764705882353
Sports : 23008.898550724636
Music : 57326.530303030304
Weather : 52279.892857142855
Entertainment : 14029.830708661417
Health & Fitness : 23298.015384615384
Games : 22788.6696905016
Social Networking : 71548.34905660378
Navigation : 86090.33333333333
Medical : 612.0


In [17]:
for app in free_ios:
    if app[-5] == "Navigation":
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [18]:
for app in free_ios:
    if app[-5] == "Reference":
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


In [34]:
for app in free_ios:
    if app[-5] == "Entertainment":
        print(app[1], ':', app[5])

Netflix : 308844
Fandango Movies - Times + Tickets : 291787
Colorfy: Coloring Book for Adults : 247809
IMDb Movies & TV - Trailers and Showtimes : 183425
TRUTH or DARE!!! - FREE : 171055
Mad Libs : 117889
Twitch : 109549
Action Movie FX : 101222
Voice Changer Plus : 98777
iFunny :) : 98344
The CW : 97368
The Moron Test : 88613
DIRECTV : 81006
ABC – Watch Live TV & Stream Full Episodes : 78890
Xbox : 72187
Redbox : 60236
Talking Tom Cat 2 for iPad : 56399
Hulu: Watch TV Shows & Stream the Latest Movies : 56170
NBC – Watch Now and Stream Full TV Episodes : 55950
Emoji> : 55338
DIRECTV App for iPad : 47506
Amazon Prime Video : 43667
CBS Full Episodes and Live TV : 39436
FOX NOW - Watch Full Episodes and Stream Live TV : 39391
Talking Angela for iPad : 32763
Recolor - Coloring Book : 31180
Talking Ben the Dog for iPad : 31116
Talking Tom Cat for iPad : 29492
YouTube Kids : 28560
Tom's Love Letters : 27711
HBO GO : 26278
NFL Sunday Ticket : 24258
Pigment - Coloring Book for Adults : 23967
D

In [32]:
for app in free_ios:
    if app[-5] == "Shopping":
        print(app[1], ':', app[5])

Groupon - Deals, Coupons & Discount Shopping App : 417779
eBay: Best App to Buy, Sell, Save! Online Shopping : 262241
Wish - Shopping Made Fun : 141960
shopkick - Shopping Rewards & Discounts : 130823
Amazon App: shop, scan, compare, and read reviews : 126312
Target : 108131
Zappos: shop shoes & clothes, fast free shipping : 103655
Walgreens – Pharmacy, Photo, Coupons and Shopping : 88885
Best Buy : 80424
Walmart: Free 2-Day Shipping,* Easy Store Shopping : 70286
OfferUp - Buy. Sell. Simple. : 57348
Apple Store : 55171
Shop Savvy Barcode Scanner - Price Compare & Deals : 54630
Ibotta: Cash Back App, Grocery Coupons & Shopping : 44313
letgo: Buy & Sell Second Hand Stuff : 38424
CVS Pharmacy : 35981
Victoria’s Secret – The Sexiest Bras & Lingerie : 34507
Etsy: Shop Handmade, Vintage & Creative Goods : 30434
Gilt : 26353
Mercari: Shopping Marketplace to Buy & Sell Stuff : 24244
Shopular Coupons, Weekly Deals for Target, Walmart : 22729
RetailMeNot Shopping Deals, Coupons, Savings : 18544


>For category like Social Networking, Music, Navigation and Entertainment which dominated by monopolists like Facebook, Spotify and Google map results in skewing the distribution and makes them impressive in paper.  

>Other category requires expertise like Finance, Food and drink, Shopping, Sport, News, Health and Fitness are not suitable to try a low cost prototype.


>App with low user engagement like weather wont be sufficient to generate profit through ads.

> Productivity and Games markets showing potential but it seems to be saturated with popular apps for a long time (fruits ninja, temple run, microsoft office etc)

**Book cateory shows much potential to add values in and it requires user to spend a lot of time in in the app. It could be an improved version of the Bible, or collection of famous classic which copyright free**

In [25]:
android_category = freq_table(free_android, 1)

for category in android_category:
    total = 0
    len_category = 0
    for app in free_android:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
        
    average_installs = total / len_category 
    print(category, ':', average_installs)

    

AUTO_AND_VEHICLES : 647317.8170731707
FAMILY : 3695641.8198090694
FINANCE : 1387692.475609756
SHOPPING : 7036877.311557789
BEAUTY : 513151.88679245283
EDUCATION : 1833495.145631068
MAPS_AND_NAVIGATION : 4056941.7741935486
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
PHOTOGRAPHY : 17840110.40229885
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
ART_AND_DESIGN : 1986335.0877192982
SPORTS : 3638640.1428571427
BUSINESS : 1712290.1474201474
FOOD_AND_DRINK : 1924897.7363636363
PARENTING : 542603.6206896552
COMMUNICATION : 38456119.167247385
WEATHER : 5074486.197183099
COMICS : 817657.2727272727
MEDICAL : 120550.61980830671
HEALTH_AND_FITNESS : 4188821.9853479853
BOOKS_AND_REFERENCE : 8767811.894736841
PERSONALIZATION : 5201482.6122448975
SOCIAL : 23253652.127118643
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
DATING : 854028.8303030303
HOUSE_AND_HOME : 1331540.5616438356
PRODUCTIVITY : 16787331.344927534
TRAVEL_AND_LOCAL : 13984077.

In [27]:
for app in free_android:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

In [30]:
for app in free_android:
    if app[1] == 'PHOTOGRAPHY' and (app[5] == '10,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '1,000,000+'
                                            or app[5] == '500,000+'):
        print(app[0], ':', app[5])

TouchNote: Cards & Gifts : 1,000,000+
FreePrints – Free Photos Delivered : 1,000,000+
Groovebook Photo Books & Gifts : 500,000+
LALALAB prints your photos, photobooks and magnets : 1,000,000+
Snapfish : 1,000,000+
HD Camera - Best Cam with filters & panorama : 5,000,000+
LightX Photo Editor & Photo Effects : 10,000,000+
Sweet Snap - live filter, Selfie photo edit : 10,000,000+
HD Camera - Quick Snap Photo & Video : 1,000,000+
Waterfall Photo Frames : 1,000,000+
Huji Cam : 5,000,000+
Unicorn Photo : 1,000,000+
HD Camera : 5,000,000+
Makeup Editor -Beauty Photo Editor & Selfie Camera : 1,000,000+
Makeup Photo Editor: Makeup Camera & Makeup Editor : 1,000,000+
Moto Photo Editor : 5,000,000+
Garden Photo Frames - Garden Photo Editor : 500,000+
Photo Frame : 10,000,000+
Sweet Snap Lite - live filter, Selfie photo editor : 500,000+
Selfie Camera: Beauty Camera, Photo Editor,Collage : 1,000,000+
Night Photo Frame : 1,000,000+
Selfie Photo Editor : 1,000,000+
Kids Photo Frames : 1,000,000+
Pen

In [31]:
for app in free_android:
    if app[1] == 'BEAUTY' and (app[5] == '10,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '1,000,000+'
                                            or app[5] == '500,000+'):
        print(app[0], ':', app[5])

Hush - Beauty for Everyone : 500,000+
ipsy: Makeup, Beauty, and Tips : 1,000,000+
BestCam Selfie-selfie, beauty camera, photo editor : 500,000+
Mirror - Zoom & Exposure - : 1,000,000+
Beauty Selfie Camera : 500,000+
Filters for Selfie : 1,000,000+
Ulta Beauty : 1,000,000+
Selfie Camera : 1,000,000+
Beauty Camera - Selfie Camera : 10,000,000+
Girls Hairstyles : 500,000+
Mirror Camera (Mirror + Selfie Camera) : 1,000,000+
Sephora: Skin Care, Beauty Makeup & Fragrance Shop : 1,000,000+
Sticker Camera - Selfie Filters, Beauty Camera : 500,000+
Filters for B Live : 500,000+
Rainbow Camera : 1,000,000+
Best Hairstyles step by step : 5,000,000+


In [37]:
for app in free_android:
    if app[1] == 'COMICS' and (app[5] == '10,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '1,000,000+'
                                            or app[5] == '500,000+'):
        print(app[0], ':', app[5])

Manga Master - Best manga & comic reader : 500,000+
GANMA! - All original stories free of charge for all original comics : 1,000,000+
Röhrich Werner Soundboard : 500,000+
Manga Rock - Best Manga Reader : 1,000,000+
comico Popular Original Cartoon Updated Everyday Comico : 5,000,000+
漫咖 Comics - Manga,Novel and Stories : 1,000,000+
Manga Zero - Japanese cartoon and comic reader : 1,000,000+
Marvel Unlimited : 1,000,000+
Tapas – Comics, Novels, and Stories : 1,000,000+
Narrator's Voice : 5,000,000+
Buff Thun - Daily Free Webtoon / Comics / Web Fiction / Mini Game : 500,000+
pixiv comic - everyone's manga app : 1,000,000+
Perfect Viewer : 5,000,000+
Memes Button : 1,000,000+
LINE WEBTOON - Free Comics : 10,000,000+
Izneo, Read Manga, Comics & BD : 500,000+
WebComics : 1,000,000+
Lezhin Comics - Daily Releases : 1,000,000+
Manga Books : 500,000+
DC Comics : 1,000,000+
Comics : 5,000,000+


In [40]:
for app in free_android:
    if app[1]== "BOOKS_AND_REFERENCE" and (app[5] == '10,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '1,000,000+'
                                            or app[5] == '500,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Only 30 days in English, the guideline is guaranteed : 500,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Azpen eReader : 500,000+
Aldiko Book Reader : 10,0

A lot of categories in